<a href="https://colab.research.google.com/github/tpessoa-dev/plataformas-cognitivas/blob/main/Concess%C3%A3o_de_Cr%C3%A9dito_com_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configurações

![alt text](https://github.com/elthonf/plataformas-cognitivas-docker/blob/master/images/capa.png?raw=1)

In [1]:
#@title Parâmetros da solução
#@markdown Informe a quantidade de clientes que devem ser analisados.
qtde_clientes = 6  #@param {type: "slider", min: 1, max: 10}

#@markdown Informe o IP do servidor de predição da sua solução:
my_server_ip = "35.222.152.218" #@param {type:"string"}

#@markdown Informe a porta do servidor de predição da sua solução, se nada foi alterado, deve ser 443:
my_server_port = "443" #@param {type:"string"}


#@markdown Ah, e não se esqueça de executar esta célula!


In [2]:
import requests
import pandas as pd
import json

# Obtenção dos empréstimos pendentes de análise

Vamos agora obter os clientes que estã precisand de empréstimo e exibí-los na tela

In [3]:
url = "https://us-central1-emf-teacher.cloudfunctions.net/function-aulas-getclient?qtde={0}".format(qtde_clientes)
headers = {'Content-Type': 'application/json'}
response = requests.request("GET", url, headers=headers)
response_content = response.text.encode('utf8').decode()
clientes = pd.read_json(response_content)

In [4]:
clientes

,nome,renda,idade,etnia,sexo,casapropria,outrasrendas,estadocivil,escolaridade
11662,"Jesse, Jones",3985.035660,55.220183,0,1,1,0,0,2
11215,"Jahlysa, Tran",3017.500093,25.750213,0,1,0,0,0,2
46709,"Meagan, Monarrez",4373.593186,43.462012,0,1,1,1,1,2
18231,"Victor, Hernandez",3665.541064,28.507155,1,0,1,1,1,2
33367,"Jasmine, Moreno",3482.393954,35.719835,0,1,1,1,1,0
7410,"Renee, Moquino",3177.975340,37.891813,1,1,1,0,1,2


# Predição

O cliente agora será submetido a dois modelos de Machine Learning preparados para a análise.

In [5]:
url = "http://{}:{}/predict".format(my_server_ip, my_server_port)
headers = {'Content-Type': 'application/json'}
conteudo = clientes.to_json()

In [6]:
response01 = requests.request("POST", url + "?model=modelo01", headers=headers, data=conteudo)
respostas01 = json.loads(response01.text.encode('utf8').decode())
respostas01

{'prediction': [0, 1, 0, 0, 0, 0],
 'proba': [[0.9228494245453985, 0.07715057545460144],
  [0.04597488365257296, 0.954025116347427],
  [0.9953121687214376, 0.004687831278562434],
  [0.9373435728672841, 0.06265642713271591],
  [0.8977153416335896, 0.10228465836641044],
  [0.7535118402624263, 0.2464881597375737]]}

In [7]:
df_01 = pd.Series(respostas01['prediction'], index = clientes.index, name="PredicaoML01")
df_01B = pd.Series(respostas01['proba'], index = clientes.index, name="PredicaoML01B")

In [8]:
response02 = requests.request("POST", url + "?model=modelo02", headers=headers, data=conteudo)
respostas02 = json.loads(response02.text.encode('utf8').decode())
respostas02

{'prediction': [0.13534460157225717,
  1.0711610170621109,
  0.02057058726599434,
  0.04166179721793638,
  0.24266802936497,
  0.27502317856286396]}

In [9]:
df_02 = pd.Series(respostas02['prediction'], index = clientes.index, name="PredicaoML02")

In [10]:
pd.DataFrame(clientes.loc[:,"nome"])\
    .merge(right= df_01, left_index=True, right_index=True)\
    .merge(df_01B, left_index=True, right_index=True)\
    .merge(df_02, left_index=True, right_index=True)

,nome,PredicaoML01,PredicaoML01B,PredicaoML02
11662,"Jesse, Jones",0,"[0.9228494245453985, 0.07715057545460144]",0.135345
11215,"Jahlysa, Tran",1,"[0.04597488365257296, 0.954025116347427]",1.071161
46709,"Meagan, Monarrez",0,"[0.9953121687214376, 0.004687831278562434]",0.020571
18231,"Victor, Hernandez",0,"[0.9373435728672841, 0.06265642713271591]",0.041662
33367,"Jasmine, Moreno",0,"[0.8977153416335896, 0.10228465836641044]",0.242668
7410,"Renee, Moquino",0,"[0.7535118402624263, 0.2464881597375737]",0.275023


# **Conclusão**



O cliente **Jahlysa, Tran** aponta com grande risco de Default e portanto terá o empréstimo negado.

O cliente **Meagan, Monarrez** apresenta o mais alto score para bom pagador e portanto pode contar com linhas de créditos diferenciadas

Os demais clientes apresentam scores positivos e podem contar com linhas de crédito padrão.